In [1]:
import os
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt
import ujson as json

from tqdm.notebook import tqdm


# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', 20)
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [2]:
PATH_TO_DATA = './data/'

#### Функция для чтения файлов с описаниями игр

Следующая функция `read_matches(filename)` может использоваться для чтения необработанных данных об играх Dota 2.

Мы рекомендуем установить два пакета Python: `ujson` и `tqdm`, это сделает выполнение быстрее и

In [3]:
def read_matches(matches_file):
    MATCHES_COUNT = {
        'test_matches.jsonl': 10000,
        'train_matches.jsonl': 39675,
    }
    _, filename = os.path.split(matches_file)
    total_matches = MATCHES_COUNT.get(filename)
    
    with open(matches_file) as fin:
        for line in tqdm(fin, total=total_matches):
            yield json.loads(line)

## Как создать начальные признаки с нуля

Теперь мы раскрываем код, который мы использовали для создания первоначальных признаков `train_features.csv` и `test_features.csv`. Вы можете изменить следующий код, чтобы добавить дополнительные функции.

В двух словах:

1. функция `extract_features_csv(match)` извлекает характеристики из описания игры и записывает их в словарь.
2. функция `extract_targets_csv(match, target)` извлекает целевую переменную `radiant_win`
3. перебирая файл с необработанными данными, собираем все признаки
4. с помощью `pandas.DataFrame.from_records()` мы создаем датафреймы с новыми признаками.

In [4]:
MATCH_FEATURES = [
    ('game_time', lambda m: m['game_time']),
    ('game_mode', lambda m: m['game_mode']),
    ('lobby_type', lambda m: m['lobby_type']),
    ('objectives_len', lambda m: len(m['objectives'])),
    ('chat_len', lambda m: len(m['chat'])),
    ('number_of_teamfights', lambda m: len(m['teamfights'])),
    
]

TEAM_FIELDS = [
    'radiant_tower_kills',
    'dire_tower_kills',
    'radiant_barrack_kills',
    'dire_barrack_kills',
    'radiant_chat',
    'dire_chat',
    'radiant_aegis',
    'dire_aegis'
    
]

PLAYER_FIELDS_A = [
    # признаки без обработки
    'hero_id',
    'hero_name',
    'obs_placed',
    'sen_placed',
    'creeps_stacked',
    'camps_stacked',
    'rune_pickups',
    'firstblood_claimed',
    'teamfight_participation',
    'towers_killed',
    'roshans_killed',
    'stuns',
    'randomed',
    'pred_vict',
    'gold',
    'lh',
    'xp',
    'x',
    'y',
    'health',
    'max_health',
    'max_mana',
    'level',
    'denies',
    'nearby_creep_death_count',
    'kills',
    'deaths',
    'assists',
    
]

PLAYER_FIELDS_B = [
    # признаки с обработкой
    'kda',
    'ability_level',
    'max_hero_hit',
    'gpm_max',
    'gpm_avg',
    'gpm_delta',
    'lhpm_max',
    'lhpm_avg',
    'lhpm_delta',
    'dnpm_max',
    'dnpm_avg',
    'dnpm_delta',
    'xppm_max',
    'xppm_avg',
    'xppm_delta',
    'purchases',
    'buybacks',
    'actions',
    'pings',
    'item_uses',
    'ability_uses',
    'hero_hits',
    'hero_damage',
    'tower_damage',
    'kill_streaks',
    'multi_kills',
    'life_state_0',
    'life_state_1',
    'life_state_2',
    'healing',
    'slots',
    'hero_inventory',
    
]


In [5]:
def extract_features_csv(match):
    row = [
        ('match_id_hash', match['match_id_hash']),
    ]
    # заполняем MATCH_FEATURES
    for field, f in MATCH_FEATURES:
        row.append((field, f(match)))
    
    # заполняем TEAM_FIELDS
    # обработка objectives
    radiant_tower_kills = 0
    radiant_barrack_kills = 0
    dire_tower_kills = 0
    dire_barrack_kills = 0
    radiant_aegis = 0
    dire_aegis = 0
    for objective in match['objectives']:
        if objective['type'] == 'CHAT_MESSAGE_TOWER_KILL':
            if objective['team'] == 2:
                radiant_tower_kills += 1
            elif objective['team'] == 3:
                dire_tower_kills += 1
        elif objective['type'] == 'CHAT_MESSAGE_TOWER_DENY':
                if objective['player_slot'] in [128, 129, 130, 131, 132]:
                    radiant_tower_kills += 1
                elif objective['player_slot'] in [0, 1, 2, 3, 4]:
                    dire_tower_kills += 1
        elif objective['type'] == 'CHAT_MESSAGE_BARRACKS_KILL':
            if objective['key'] in ['1', '2', '4', '8', '16', '32']:
                radiant_barrack_kills += 1
            elif objective['key'] in ['64', '128', '256', '512', '1024', '2048']:
                dire_barrack_kills += 1
        elif objective['type'] == 'CHAT_MESSAGE_AEGIS':
                if objective['player_slot'] in [0, 1, 2, 3, 4]:
                    radiant_aegis += 1
                elif objective['player_slot'] in [128, 129, 130, 131, 132]: 
                    dire_aegis += 1
    row.append(('radiant_tower_kills', radiant_tower_kills))
    row.append(('dire_tower_kills', dire_tower_kills))
    row.append(('radiant_barrack_kills', radiant_barrack_kills))
    row.append(('dire_barrack_kills', dire_barrack_kills))
    row.append(('radiant_aegis', radiant_aegis))
    row.append(('dire_aegis', dire_aegis))
    # обработка chat
    radiant_chat = 0
    dire_chat= 0
    for message in match['chat']:
        if message['player_slot'] in [0, 1, 2, 3, 4]:
            radiant_chat += 1
        elif message['player_slot'] in [128, 129, 130, 131, 132]:
            dire_chat += 1
    row.append(('radiant_chat', radiant_chat))
    row.append(('dire_chat', dire_chat))
                    
    # заполняем PLAYER_FIELDS
    for slot, player in enumerate(match['players']):
        if slot < 5:
            player_name = 'r%d' % (slot + 1)
        else:
            player_name = 'd%d' % (slot - 4)
        
        # заполняем PLAYER_FIELDS_A
        for field in PLAYER_FIELDS_A:
            column_name = '%s_%s' % (player_name, field)
            row.append((column_name, player[field]))
    
        # заполняем PLAYER_FIELDS_B
        times = player['times']
        len_t = len(times)
        for field in PLAYER_FIELDS_B:
            column_name = '%s_%s' % (player_name, field)
            
            if field == 'kda':
                if player['deaths'] != 0:
                    temp = (player['kills'] + player['assists']) / player['deaths']
                else:
                    temp = (player['kills'] + player['assists'])
                row.append((column_name, temp))
            elif field == 'ability_level':
                temp = len(player['ability_upgrades'])
                row.append((column_name, temp))
            elif field == 'max_hero_hit':
                temp = player['max_hero_hit']['value']
                row.append((column_name, temp))
            elif field == 'gpm_max':
                if player['gold_t'] and times:
                    try:
                        temp = player['gold_t'][-1] / (times[-1] / 60) 
                        row.append((column_name, temp))
                    except ZeroDivisionError:
                        row.append((column_name, 0))
                else:
                    row.append((column_name, 0))
            elif field == 'gpm_avg':
                if player['gold_t'] and times:
                    a = np.array(player['gold_t'][1:], dtype=float)
                    b = np.divide(times[1:], 60.0)
                    temp = np.mean(np.divide(a, b, out=np.zeros_like(a), where=b!=0))
                    row.append((column_name, temp))
                else:
                    row.append((column_name, 0))
            elif field == 'gpm_delta':
                if len(player['gold_t']) > 1:
                    sum_ = 0
                    a = player['gold_t']
                    for i in range(1, len(times)):
                        delta_a = a[i] - a[i - 1]
                        delta_t = times[i] - times[i - 1]
                        sum_ += delta_a / delta_t
                    try:
                        temp = sum_ / (len(times) - 1)
                    except ZeroDivisionError:
                        temp = sum_ / 1
                else:
                    temp = 0
                row.append((column_name, temp))
            elif field == 'lhpm_max':
                if player['lh_t'] and times:
                    try:
                        temp = player['lh_t'][-1] / (times[-1] / 60) 
                        row.append((column_name, temp))
                    except ZeroDivisionError:
                        row.append((column_name, 0))
                else:
                    row.append((column_name, 0))
            elif field == 'lhpm_avg':
                if player['lh_t'] and times:
                    a = np.array(player['lh_t'][1:], dtype=float)
                    b = np.divide(times[1:], 60.0)
                    temp = np.mean(np.divide(a, b, out=np.zeros_like(a), where=b!=0))
                    row.append((column_name, temp))
                else:
                    row.append((column_name, 0))
            elif field == 'lhpm_delta':
                if len(player['lh_t']) > 1:
                    sum_ = 0
                    a = player['lh_t']
                    for i in range(1, len(times)):
                        delta_a = a[i] - a[i - 1]
                        delta_t = times[i] - times[i - 1]
                        sum_ += delta_a / delta_t
                    try:
                        temp = sum_ / (len(times) - 1)
                    except ZeroDivisionError:
                        temp = sum_ / 1
                else:
                    temp = 0
                row.append((column_name, temp))
            elif field == 'dnpm_max':
                if player['dn_t'] and times:
                    try:
                        temp = player['dn_t'][-1] / (times[-1] / 60) 
                        row.append((column_name, temp))
                    except ZeroDivisionError:
                        row.append((column_name, 0))
                else:
                    row.append((column_name, 0))
            elif field == 'dnpm_avg':
                if player['dn_t'] and times:
                    a = np.array(player['dn_t'][1:], dtype=float)
                    b = np.divide(times[1:], 60.0)
                    temp = np.mean(np.divide(a, b, out=np.zeros_like(a), where=b!=0))
                    row.append((column_name, temp))
                else:
                    row.append((column_name, 0))
            elif field == 'dnpm_delta':
                if len(player['dn_t']) > 1:
                    sum_ = 0
                    a = player['dn_t']
                    for i in range(1, len(times)):
                        delta_a = a[i] - a[i - 1]
                        delta_t = times[i] - times[i - 1]
                        sum_ += delta_a / delta_t
                    try:
                        temp = sum_ / (len(times) - 1)
                    except ZeroDivisionError:
                        temp = sum_ / 1
                else:
                    temp = 0
                row.append((column_name, temp))
            elif field == 'xppm_max':
                if player['xp_t'] and times:
                    try:
                        temp = player['xp_t'][-1] / (times[-1] / 60) 
                        row.append((column_name, temp))
                    except ZeroDivisionError:
                        row.append((column_name, 0))
                else:
                    row.append((column_name, 0))
            elif field == 'xppm_avg':
                if player['xp_t'] and times:
                    a = np.array(player['xp_t'][1:], dtype=float)
                    b = np.divide(times[1:], 60.0)
                    temp = np.mean(np.divide(a, b, out=np.zeros_like(a), where=b!=0))
                    row.append((column_name, temp))
                else:
                    row.append((column_name, 0))
            elif field == 'xppm_delta':
                if len(player['xp_t']) > 1:
                    sum_ = 0
                    a = player['xp_t']
                    for i in range(1, len(times)):
                        delta_a = a[i] - a[i - 1]
                        delta_t = times[i] - times[i - 1]
                        sum_ += delta_a / delta_t
                    try:
                        temp = sum_ / (len(times) - 1)
                    except ZeroDivisionError:
                        temp = sum_ / 1
                else:
                    temp = 0
                row.append((column_name, temp))
            elif field == 'purchases':
                temp = len(player['purchase_log'])
                row.append((column_name, temp))
            elif field == 'buybacks':
                temp = len(player['buyback_log'])
                row.append((column_name, temp))
            elif field == 'actions':
                temp = sum(player['actions'].values())
                row.append((column_name, temp))
            elif field == 'pings':
                temp = sum(player['pings'].values())
                row.append((column_name, temp))
            elif field == 'item_uses':
                temp = sum(player['item_uses'].values())
                row.append((column_name, temp))
            elif field == 'ability_uses':
                temp = sum(player['ability_uses'].values())
                row.append((column_name, temp))
            elif field == 'hero_hits':
                temp = sum(player['hero_hits'].values())
                row.append((column_name, temp))
            elif field == 'hero_damage':
                damage_dict = player['damage']
                keys = list(damage_dict.keys())
                temp = 0
                for key in keys:
                    if 'hero' in key:
                        temp += damage_dict[key]
                row.append((column_name, temp))
            elif field == 'tower_damage':
                damage_dict = player['damage']
                keys = list(damage_dict.keys())
                temp = 0
                for key in keys:
                    if 'tower' in key:
                        temp += damage_dict[key]
                row.append((column_name, temp))
            elif field == 'kill_streaks':
                temp = sum(player['kill_streaks'].values())
                row.append((column_name, temp))
            elif field == 'multi_kills':
                temp = sum(player['multi_kills'].values())
                row.append((column_name, temp))
            elif field == 'life_state_0':
                life_state_dict = player['life_state']
                try:
                    temp = life_state_dict['0']
                except KeyError:
                    temp = 0
                row.append((column_name, temp))
            elif field == 'life_state_1':
                life_state_dict = player['life_state']
                try:
                    temp = life_state_dict['1']
                except KeyError:
                    temp = 0
                row.append((column_name, temp))
            elif field == 'life_state_2':
                life_state_dict = player['life_state']
                try:
                    temp = life_state_dict['2']
                except KeyError:
                    temp = 0
                row.append((column_name, temp))
            elif field == 'healing':
                temp = sum(player['healing'].values())
                row.append((column_name, temp))
            elif field == 'slots':
                temp = len(player['hero_inventory'])
                row.append((column_name, temp))
            elif field == 'hero_inventory':
                h_inv = player['hero_inventory']
                p_inv = []
                for h in h_inv:
                    p_inv.append(h['id'])
                if p_inv:
                    temp = ' '.join(p_inv)
                else:
                    temp = 'no_items'
                row.append((column_name, temp))
                
    
            
    return collections.OrderedDict(row)
    
def extract_targets_csv(match, targets):
    return collections.OrderedDict([('match_id_hash', match['match_id_hash'])] + [
        (field, targets[field])
        for field in ['game_time', 'radiant_win', 'duration', 'time_remaining', 'next_roshan_team']
    ])

In [6]:
%%time

df_new_features = []
df_new_targets = []

for match in read_matches(os.path.join(PATH_TO_DATA, 'train_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv(match)
    targets = extract_targets_csv(match, match['targets'])
    
    df_new_features.append(features)
    df_new_targets.append(targets)
   

  0%|          | 0/39675 [00:00<?, ?it/s]

c:\users\eugene\appdata\local\programs\python\python39\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\eugene\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Wall time: 2min 23s


In [7]:
df_new_features = pd.DataFrame.from_records(df_new_features).set_index('match_id_hash')
df_new_targets = pd.DataFrame.from_records(df_new_targets).set_index('match_id_hash')

In [8]:
df_new_features = df_new_features.fillna(0)

In [9]:
print(df_new_features.shape)
df_new_features

(39675, 614)


,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,radiant_tower_kills,dire_tower_kills,radiant_barrack_kills,dire_barrack_kills,radiant_aegis,dire_aegis,radiant_chat,dire_chat,r1_hero_id,r1_hero_name,r1_obs_placed,r1_sen_placed,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_stuns,r1_randomed,r1_pred_vict,r1_gold,r1_lh,r1_xp,r1_x,r1_y,r1_health,r1_max_health,r1_max_mana,r1_level,r1_denies,r1_nearby_creep_death_count,r1_kills,r1_deaths,r1_assists,r1_kda,r1_ability_level,r1_max_hero_hit,r1_gpm_max,r1_gpm_avg,r1_gpm_delta,r1_lhpm_max,r1_lhpm_avg,r1_lhpm_delta,r1_dnpm_max,r1_dnpm_avg,r1_dnpm_delta,r1_xppm_max,r1_xppm_avg,r1_xppm_delta,r1_purchases,r1_buybacks,r1_actions,r1_pings,r1_item_uses,r1_ability_uses,r1_hero_hits,r1_hero_damage,r1_tower_damage,r1_kill_streaks,r1_multi_kills,r1_life_state_0,r1_life_state_1,r1_life_state_2,r1_healing,r1_slots,r1_hero_inventory,r2_hero_id,r2_hero_name,r2_obs_placed,r2_sen_placed,r2_creeps_stacked,r2_camps_stacked,r2_rune_pickups,r2_firstblood_claimed,r2_teamfight_participation,r2_towers_killed,r2_roshans_killed,r2_stuns,r2_randomed,r2_pred_vict,r2_gold,r2_lh,r2_xp,r2_x,r2_y,r2_health,r2_max_health,r2_max_mana,r2_level,r2_denies,r2_nearby_creep_death_count,r2_kills,r2_deaths,r2_assists,r2_kda,r2_ability_level,r2_max_hero_hit,r2_gpm_max,r2_gpm_avg,r2_gpm_delta,r2_lhpm_max,r2_lhpm_avg,r2_lhpm_delta,r2_dnpm_max,r2_dnpm_avg,r2_dnpm_delta,r2_xppm_max,r2_xppm_avg,r2_xppm_delta,r2_purchases,r2_buybacks,r2_actions,r2_pings,r2_item_uses,r2_ability_uses,r2_hero_hits,r2_hero_damage,r2_tower_damage,r2_kill_streaks,r2_multi_kills,r2_life_state_0,r2_life_state_1,r2_life_state_2,r2_healing,r2_slots,r2_hero_inventory,r3_hero_id,r3_hero_name,r3_obs_placed,r3_sen_placed,r3_creeps_stacked,r3_camps_stacked,r3_rune_pickups,r3_firstblood_claimed,r3_teamfight_participation,r3_towers_killed,r3_roshans_killed,r3_stuns,r3_randomed,r3_pred_vict,r3_gold,r3_lh,r3_xp,r3_x,r3_y,r3_health,r3_max_health,r3_max_mana,r3_level,r3_denies,r3_nearby_creep_death_count,r3_kills,r3_deaths,r3_assists,r3_kda,r3_ability_level,r3_max_hero_hit,r3_gpm_max,r3_gpm_avg,r3_gpm_delta,r3_lhpm_max,r3_lhpm_avg,r3_lhpm_delta,r3_dnpm_max,r3_dnpm_avg,r3_dnpm_delta,r3_xppm_max,r3_xppm_avg,r3_xppm_delta,r3_purchases,r3_buybacks,r3_actions,r3_pings,r3_item_uses,r3_ability_uses,r3_hero_hits,r3_hero_damage,r3_tower_damage,r3_kill_streaks,r3_multi_kills,r3_life_state_0,r3_life_state_1,r3_life_state_2,r3_healing,r3_slots,r3_hero_inventory,r4_hero_id,r4_hero_name,r4_obs_placed,r4_sen_placed,r4_creeps_stacked,r4_camps_stacked,r4_rune_pickups,r4_firstblood_claimed,r4_teamfight_participation,r4_towers_killed,r4_roshans_killed,r4_stuns,r4_randomed,r4_pred_vict,r4_gold,r4_lh,r4_xp,r4_x,r4_y,r4_health,r4_max_health,r4_max_mana,r4_level,r4_denies,r4_nearby_creep_death_count,r4_kills,r4_deaths,r4_assists,r4_kda,r4_ability_level,r4_max_hero_hit,r4_gpm_max,r4_gpm_avg,r4_gpm_delta,r4_lhpm_max,r4_lhpm_avg,r4_lhpm_delta,r4_dnpm_max,r4_dnpm_avg,r4_dnpm_delta,r4_xppm_max,r4_xppm_avg,r4_xppm_delta,r4_purchases,r4_buybacks,r4_actions,r4_pings,r4_item_uses,r4_ability_uses,r4_hero_hits,r4_hero_damage,r4_tower_damage,r4_kill_streaks,r4_multi_kills,r4_life_state_0,r4_life_state_1,r4_life_state_2,r4_healing,r4_slots,r4_hero_inventory,r5_hero_id,r5_hero_name,r5_obs_placed,r5_sen_placed,r5_creeps_stacked,r5_camps_stacked,r5_rune_pickups,r5_firstblood_claimed,r5_teamfight_participation,r5_towers_killed,r5_roshans_killed,r5_stuns,r5_randomed,r5_pred_vict,r5_gold,r5_lh,r5_xp,r5_x,r5_y,r5_health,r5_max_health,r5_max_mana,r5_level,r5_denies,r5_nearby_creep_death_count,r5_kills,r5_deaths,r5_assists,r5_kda,r5_ability_level,r5_max_hero_hit,r5_gpm_max,r5_gpm_avg,r5_gpm_delta,r5_lhpm_max,r5_lhpm_avg,r5_lhpm_delta,r5_dnpm_max,r5_dnpm_avg,r5_dnpm_delta,r5_xppm_max,r5_xppm_avg,r5_xppm_delta,r5_purchases,r5_buybacks,r5_actions,r5_pings,r5_item_uses,r5_ability_uses,r5_hero_hits,r5_he

In [10]:
print(df_new_targets.shape)
df_new_targets

(39675, 5)


,game_time,radiant_win,duration,time_remaining,next_roshan_team
match_id_hash,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,False,992,837,None
b9c57c450ce74a2af79c9ce96fac144d,658,True,1154,496,None
6db558535151ea18ca70a6892197db41,21,True,1503,1482,Radiant
46a0ddce8f7ed2a8d9bd5edcbb925682,576,True,1952,1376,None
b1b35ff97723d9b7ade1c9c3cf48f770,453,False,2001,1548,None
...,...,...,...,...,...
defd0caeed6ea83d7d5fbdec013fe7d1,1783,False,2766,983,Dire
bc7a87ed5f9c2bca55f9f7a93da0b0c5,377,True,3107,2730,Radiant
e2ca68ac1a6847f4a37f6c9c8ee8695b,643,False,1804,1161,None


In [11]:
df_new_features['radiant_win'] = df_new_targets['radiant_win'].values
df_new_features

,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,radiant_tower_kills,dire_tower_kills,radiant_barrack_kills,dire_barrack_kills,radiant_aegis,dire_aegis,radiant_chat,dire_chat,r1_hero_id,r1_hero_name,r1_obs_placed,r1_sen_placed,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_stuns,r1_randomed,r1_pred_vict,r1_gold,r1_lh,r1_xp,r1_x,r1_y,r1_health,r1_max_health,r1_max_mana,r1_level,r1_denies,r1_nearby_creep_death_count,r1_kills,r1_deaths,r1_assists,r1_kda,r1_ability_level,r1_max_hero_hit,r1_gpm_max,r1_gpm_avg,r1_gpm_delta,r1_lhpm_max,r1_lhpm_avg,r1_lhpm_delta,r1_dnpm_max,r1_dnpm_avg,r1_dnpm_delta,r1_xppm_max,r1_xppm_avg,r1_xppm_delta,r1_purchases,r1_buybacks,r1_actions,r1_pings,r1_item_uses,r1_ability_uses,r1_hero_hits,r1_hero_damage,r1_tower_damage,r1_kill_streaks,r1_multi_kills,r1_life_state_0,r1_life_state_1,r1_life_state_2,r1_healing,r1_slots,r1_hero_inventory,r2_hero_id,r2_hero_name,r2_obs_placed,r2_sen_placed,r2_creeps_stacked,r2_camps_stacked,r2_rune_pickups,r2_firstblood_claimed,r2_teamfight_participation,r2_towers_killed,r2_roshans_killed,r2_stuns,r2_randomed,r2_pred_vict,r2_gold,r2_lh,r2_xp,r2_x,r2_y,r2_health,r2_max_health,r2_max_mana,r2_level,r2_denies,r2_nearby_creep_death_count,r2_kills,r2_deaths,r2_assists,r2_kda,r2_ability_level,r2_max_hero_hit,r2_gpm_max,r2_gpm_avg,r2_gpm_delta,r2_lhpm_max,r2_lhpm_avg,r2_lhpm_delta,r2_dnpm_max,r2_dnpm_avg,r2_dnpm_delta,r2_xppm_max,r2_xppm_avg,r2_xppm_delta,r2_purchases,r2_buybacks,r2_actions,r2_pings,r2_item_uses,r2_ability_uses,r2_hero_hits,r2_hero_damage,r2_tower_damage,r2_kill_streaks,r2_multi_kills,r2_life_state_0,r2_life_state_1,r2_life_state_2,r2_healing,r2_slots,r2_hero_inventory,r3_hero_id,r3_hero_name,r3_obs_placed,r3_sen_placed,r3_creeps_stacked,r3_camps_stacked,r3_rune_pickups,r3_firstblood_claimed,r3_teamfight_participation,r3_towers_killed,r3_roshans_killed,r3_stuns,r3_randomed,r3_pred_vict,r3_gold,r3_lh,r3_xp,r3_x,r3_y,r3_health,r3_max_health,r3_max_mana,r3_level,r3_denies,r3_nearby_creep_death_count,r3_kills,r3_deaths,r3_assists,r3_kda,r3_ability_level,r3_max_hero_hit,r3_gpm_max,r3_gpm_avg,r3_gpm_delta,r3_lhpm_max,r3_lhpm_avg,r3_lhpm_delta,r3_dnpm_max,r3_dnpm_avg,r3_dnpm_delta,r3_xppm_max,r3_xppm_avg,r3_xppm_delta,r3_purchases,r3_buybacks,r3_actions,r3_pings,r3_item_uses,r3_ability_uses,r3_hero_hits,r3_hero_damage,r3_tower_damage,r3_kill_streaks,r3_multi_kills,r3_life_state_0,r3_life_state_1,r3_life_state_2,r3_healing,r3_slots,r3_hero_inventory,r4_hero_id,r4_hero_name,r4_obs_placed,r4_sen_placed,r4_creeps_stacked,r4_camps_stacked,r4_rune_pickups,r4_firstblood_claimed,r4_teamfight_participation,r4_towers_killed,r4_roshans_killed,r4_stuns,r4_randomed,r4_pred_vict,r4_gold,r4_lh,r4_xp,r4_x,r4_y,r4_health,r4_max_health,r4_max_mana,r4_level,r4_denies,r4_nearby_creep_death_count,r4_kills,r4_deaths,r4_assists,r4_kda,r4_ability_level,r4_max_hero_hit,r4_gpm_max,r4_gpm_avg,r4_gpm_delta,r4_lhpm_max,r4_lhpm_avg,r4_lhpm_delta,r4_dnpm_max,r4_dnpm_avg,r4_dnpm_delta,r4_xppm_max,r4_xppm_avg,r4_xppm_delta,r4_purchases,r4_buybacks,r4_actions,r4_pings,r4_item_uses,r4_ability_uses,r4_hero_hits,r4_hero_damage,r4_tower_damage,r4_kill_streaks,r4_multi_kills,r4_life_state_0,r4_life_state_1,r4_life_state_2,r4_healing,r4_slots,r4_hero_inventory,r5_hero_id,r5_hero_name,r5_obs_placed,r5_sen_placed,r5_creeps_stacked,r5_camps_stacked,r5_rune_pickups,r5_firstblood_claimed,r5_teamfight_participation,r5_towers_killed,r5_roshans_killed,r5_stuns,r5_randomed,r5_pred_vict,r5_gold,r5_lh,r5_xp,r5_x,r5_y,r5_health,r5_max_health,r5_max_mana,r5_level,r5_denies,r5_nearby_creep_death_count,r5_kills,r5_deaths,r5_assists,r5_kda,r5_ability_level,r5_max_hero_hit,r5_gpm_max,r5_gpm_avg,r5_gpm_delta,r5_lhpm_max,r5_lhpm_avg,r5_lhpm_delta,r5_dnpm_max,r5_dnpm_avg,r5_dnpm_delta,r5_xppm_max,r5_xppm_avg,r5_xppm_delta,r5_purchases,r5_buybacks,r5_actions,r5_pings,r5_item_uses,r5_ability_uses,r5_hero_hits,r5_he

In [12]:
bool_features = ['r1_randomed',  'r2_randomed', 'r3_randomed', 'r4_randomed', 'r5_randomed', 'd1_randomed', 'd2_randomed', 'd3_randomed', 'd4_randomed', 'd5_randomed', 'r1_pred_vict',  'r2_pred_vict', 'r3_pred_vict', 'r4_pred_vict', 'r5_pred_vict', 'd1_pred_vict', 'd2_pred_vict', 'd3_pred_vict', 'd4_pred_vict', 'd5_pred_vict', 'radiant_win']
df_new_features[bool_features] = df_new_features[bool_features].astype(int)
df_new_features

,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,radiant_tower_kills,dire_tower_kills,radiant_barrack_kills,dire_barrack_kills,radiant_aegis,dire_aegis,radiant_chat,dire_chat,r1_hero_id,r1_hero_name,r1_obs_placed,r1_sen_placed,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_stuns,r1_randomed,r1_pred_vict,r1_gold,r1_lh,r1_xp,r1_x,r1_y,r1_health,r1_max_health,r1_max_mana,r1_level,r1_denies,r1_nearby_creep_death_count,r1_kills,r1_deaths,r1_assists,r1_kda,r1_ability_level,r1_max_hero_hit,r1_gpm_max,r1_gpm_avg,r1_gpm_delta,r1_lhpm_max,r1_lhpm_avg,r1_lhpm_delta,r1_dnpm_max,r1_dnpm_avg,r1_dnpm_delta,r1_xppm_max,r1_xppm_avg,r1_xppm_delta,r1_purchases,r1_buybacks,r1_actions,r1_pings,r1_item_uses,r1_ability_uses,r1_hero_hits,r1_hero_damage,r1_tower_damage,r1_kill_streaks,r1_multi_kills,r1_life_state_0,r1_life_state_1,r1_life_state_2,r1_healing,r1_slots,r1_hero_inventory,r2_hero_id,r2_hero_name,r2_obs_placed,r2_sen_placed,r2_creeps_stacked,r2_camps_stacked,r2_rune_pickups,r2_firstblood_claimed,r2_teamfight_participation,r2_towers_killed,r2_roshans_killed,r2_stuns,r2_randomed,r2_pred_vict,r2_gold,r2_lh,r2_xp,r2_x,r2_y,r2_health,r2_max_health,r2_max_mana,r2_level,r2_denies,r2_nearby_creep_death_count,r2_kills,r2_deaths,r2_assists,r2_kda,r2_ability_level,r2_max_hero_hit,r2_gpm_max,r2_gpm_avg,r2_gpm_delta,r2_lhpm_max,r2_lhpm_avg,r2_lhpm_delta,r2_dnpm_max,r2_dnpm_avg,r2_dnpm_delta,r2_xppm_max,r2_xppm_avg,r2_xppm_delta,r2_purchases,r2_buybacks,r2_actions,r2_pings,r2_item_uses,r2_ability_uses,r2_hero_hits,r2_hero_damage,r2_tower_damage,r2_kill_streaks,r2_multi_kills,r2_life_state_0,r2_life_state_1,r2_life_state_2,r2_healing,r2_slots,r2_hero_inventory,r3_hero_id,r3_hero_name,r3_obs_placed,r3_sen_placed,r3_creeps_stacked,r3_camps_stacked,r3_rune_pickups,r3_firstblood_claimed,r3_teamfight_participation,r3_towers_killed,r3_roshans_killed,r3_stuns,r3_randomed,r3_pred_vict,r3_gold,r3_lh,r3_xp,r3_x,r3_y,r3_health,r3_max_health,r3_max_mana,r3_level,r3_denies,r3_nearby_creep_death_count,r3_kills,r3_deaths,r3_assists,r3_kda,r3_ability_level,r3_max_hero_hit,r3_gpm_max,r3_gpm_avg,r3_gpm_delta,r3_lhpm_max,r3_lhpm_avg,r3_lhpm_delta,r3_dnpm_max,r3_dnpm_avg,r3_dnpm_delta,r3_xppm_max,r3_xppm_avg,r3_xppm_delta,r3_purchases,r3_buybacks,r3_actions,r3_pings,r3_item_uses,r3_ability_uses,r3_hero_hits,r3_hero_damage,r3_tower_damage,r3_kill_streaks,r3_multi_kills,r3_life_state_0,r3_life_state_1,r3_life_state_2,r3_healing,r3_slots,r3_hero_inventory,r4_hero_id,r4_hero_name,r4_obs_placed,r4_sen_placed,r4_creeps_stacked,r4_camps_stacked,r4_rune_pickups,r4_firstblood_claimed,r4_teamfight_participation,r4_towers_killed,r4_roshans_killed,r4_stuns,r4_randomed,r4_pred_vict,r4_gold,r4_lh,r4_xp,r4_x,r4_y,r4_health,r4_max_health,r4_max_mana,r4_level,r4_denies,r4_nearby_creep_death_count,r4_kills,r4_deaths,r4_assists,r4_kda,r4_ability_level,r4_max_hero_hit,r4_gpm_max,r4_gpm_avg,r4_gpm_delta,r4_lhpm_max,r4_lhpm_avg,r4_lhpm_delta,r4_dnpm_max,r4_dnpm_avg,r4_dnpm_delta,r4_xppm_max,r4_xppm_avg,r4_xppm_delta,r4_purchases,r4_buybacks,r4_actions,r4_pings,r4_item_uses,r4_ability_uses,r4_hero_hits,r4_hero_damage,r4_tower_damage,r4_kill_streaks,r4_multi_kills,r4_life_state_0,r4_life_state_1,r4_life_state_2,r4_healing,r4_slots,r4_hero_inventory,r5_hero_id,r5_hero_name,r5_obs_placed,r5_sen_placed,r5_creeps_stacked,r5_camps_stacked,r5_rune_pickups,r5_firstblood_claimed,r5_teamfight_participation,r5_towers_killed,r5_roshans_killed,r5_stuns,r5_randomed,r5_pred_vict,r5_gold,r5_lh,r5_xp,r5_x,r5_y,r5_health,r5_max_health,r5_max_mana,r5_level,r5_denies,r5_nearby_creep_death_count,r5_kills,r5_deaths,r5_assists,r5_kda,r5_ability_level,r5_max_hero_hit,r5_gpm_max,r5_gpm_avg,r5_gpm_delta,r5_lhpm_max,r5_lhpm_avg,r5_lhpm_delta,r5_dnpm_max,r5_dnpm_avg,r5_dnpm_delta,r5_xppm_max,r5_xppm_avg,r5_xppm_delta,r5_purchases,r5_buybacks,r5_actions,r5_pings,r5_item_uses,r5_ability_uses,r5_hero_hits,r5_he

In [13]:
df_new_features.to_csv('train_data.csv')

In [14]:
%%time

df_test_features = []

for match in read_matches(os.path.join(PATH_TO_DATA, 'test_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv(match)
    df_test_features.append(features)

  0%|          | 0/10000 [00:00<?, ?it/s]

c:\users\eugene\appdata\local\programs\python\python39\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\eugene\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Wall time: 33.3 s


In [15]:
df_test_features = pd.DataFrame.from_records(df_test_features).set_index('match_id_hash')

In [16]:
df_test_features = df_test_features.fillna(0)

In [17]:
print(df_test_features.shape)
df_test_features

(10000, 614)


,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,radiant_tower_kills,dire_tower_kills,radiant_barrack_kills,dire_barrack_kills,radiant_aegis,dire_aegis,radiant_chat,dire_chat,r1_hero_id,r1_hero_name,r1_obs_placed,r1_sen_placed,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_stuns,r1_randomed,r1_pred_vict,r1_gold,r1_lh,r1_xp,r1_x,r1_y,r1_health,r1_max_health,r1_max_mana,r1_level,r1_denies,r1_nearby_creep_death_count,r1_kills,r1_deaths,r1_assists,r1_kda,r1_ability_level,r1_max_hero_hit,r1_gpm_max,r1_gpm_avg,r1_gpm_delta,r1_lhpm_max,r1_lhpm_avg,r1_lhpm_delta,r1_dnpm_max,r1_dnpm_avg,r1_dnpm_delta,r1_xppm_max,r1_xppm_avg,r1_xppm_delta,r1_purchases,r1_buybacks,r1_actions,r1_pings,r1_item_uses,r1_ability_uses,r1_hero_hits,r1_hero_damage,r1_tower_damage,r1_kill_streaks,r1_multi_kills,r1_life_state_0,r1_life_state_1,r1_life_state_2,r1_healing,r1_slots,r1_hero_inventory,r2_hero_id,r2_hero_name,r2_obs_placed,r2_sen_placed,r2_creeps_stacked,r2_camps_stacked,r2_rune_pickups,r2_firstblood_claimed,r2_teamfight_participation,r2_towers_killed,r2_roshans_killed,r2_stuns,r2_randomed,r2_pred_vict,r2_gold,r2_lh,r2_xp,r2_x,r2_y,r2_health,r2_max_health,r2_max_mana,r2_level,r2_denies,r2_nearby_creep_death_count,r2_kills,r2_deaths,r2_assists,r2_kda,r2_ability_level,r2_max_hero_hit,r2_gpm_max,r2_gpm_avg,r2_gpm_delta,r2_lhpm_max,r2_lhpm_avg,r2_lhpm_delta,r2_dnpm_max,r2_dnpm_avg,r2_dnpm_delta,r2_xppm_max,r2_xppm_avg,r2_xppm_delta,r2_purchases,r2_buybacks,r2_actions,r2_pings,r2_item_uses,r2_ability_uses,r2_hero_hits,r2_hero_damage,r2_tower_damage,r2_kill_streaks,r2_multi_kills,r2_life_state_0,r2_life_state_1,r2_life_state_2,r2_healing,r2_slots,r2_hero_inventory,r3_hero_id,r3_hero_name,r3_obs_placed,r3_sen_placed,r3_creeps_stacked,r3_camps_stacked,r3_rune_pickups,r3_firstblood_claimed,r3_teamfight_participation,r3_towers_killed,r3_roshans_killed,r3_stuns,r3_randomed,r3_pred_vict,r3_gold,r3_lh,r3_xp,r3_x,r3_y,r3_health,r3_max_health,r3_max_mana,r3_level,r3_denies,r3_nearby_creep_death_count,r3_kills,r3_deaths,r3_assists,r3_kda,r3_ability_level,r3_max_hero_hit,r3_gpm_max,r3_gpm_avg,r3_gpm_delta,r3_lhpm_max,r3_lhpm_avg,r3_lhpm_delta,r3_dnpm_max,r3_dnpm_avg,r3_dnpm_delta,r3_xppm_max,r3_xppm_avg,r3_xppm_delta,r3_purchases,r3_buybacks,r3_actions,r3_pings,r3_item_uses,r3_ability_uses,r3_hero_hits,r3_hero_damage,r3_tower_damage,r3_kill_streaks,r3_multi_kills,r3_life_state_0,r3_life_state_1,r3_life_state_2,r3_healing,r3_slots,r3_hero_inventory,r4_hero_id,r4_hero_name,r4_obs_placed,r4_sen_placed,r4_creeps_stacked,r4_camps_stacked,r4_rune_pickups,r4_firstblood_claimed,r4_teamfight_participation,r4_towers_killed,r4_roshans_killed,r4_stuns,r4_randomed,r4_pred_vict,r4_gold,r4_lh,r4_xp,r4_x,r4_y,r4_health,r4_max_health,r4_max_mana,r4_level,r4_denies,r4_nearby_creep_death_count,r4_kills,r4_deaths,r4_assists,r4_kda,r4_ability_level,r4_max_hero_hit,r4_gpm_max,r4_gpm_avg,r4_gpm_delta,r4_lhpm_max,r4_lhpm_avg,r4_lhpm_delta,r4_dnpm_max,r4_dnpm_avg,r4_dnpm_delta,r4_xppm_max,r4_xppm_avg,r4_xppm_delta,r4_purchases,r4_buybacks,r4_actions,r4_pings,r4_item_uses,r4_ability_uses,r4_hero_hits,r4_hero_damage,r4_tower_damage,r4_kill_streaks,r4_multi_kills,r4_life_state_0,r4_life_state_1,r4_life_state_2,r4_healing,r4_slots,r4_hero_inventory,r5_hero_id,r5_hero_name,r5_obs_placed,r5_sen_placed,r5_creeps_stacked,r5_camps_stacked,r5_rune_pickups,r5_firstblood_claimed,r5_teamfight_participation,r5_towers_killed,r5_roshans_killed,r5_stuns,r5_randomed,r5_pred_vict,r5_gold,r5_lh,r5_xp,r5_x,r5_y,r5_health,r5_max_health,r5_max_mana,r5_level,r5_denies,r5_nearby_creep_death_count,r5_kills,r5_deaths,r5_assists,r5_kda,r5_ability_level,r5_max_hero_hit,r5_gpm_max,r5_gpm_avg,r5_gpm_delta,r5_lhpm_max,r5_lhpm_avg,r5_lhpm_delta,r5_dnpm_max,r5_dnpm_avg,r5_dnpm_delta,r5_xppm_max,r5_xppm_avg,r5_xppm_delta,r5_purchases,r5_buybacks,r5_actions,r5_pings,r5_item_uses,r5_ability_uses,r5_hero_hits,r5_he

In [18]:
bool_features = ['r1_randomed',  'r2_randomed', 'r3_randomed', 'r4_randomed', 'r5_randomed', 'd1_randomed', 'd2_randomed', 'd3_randomed', 'd4_randomed', 'd5_randomed', 'r1_pred_vict',  'r2_pred_vict', 'r3_pred_vict', 'r4_pred_vict', 'r5_pred_vict', 'd1_pred_vict', 'd2_pred_vict', 'd3_pred_vict', 'd4_pred_vict', 'd5_pred_vict']
df_test_features[bool_features] = df_test_features[bool_features].astype(int)
df_test_features

,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,radiant_tower_kills,dire_tower_kills,radiant_barrack_kills,dire_barrack_kills,radiant_aegis,dire_aegis,radiant_chat,dire_chat,r1_hero_id,r1_hero_name,r1_obs_placed,r1_sen_placed,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_stuns,r1_randomed,r1_pred_vict,r1_gold,r1_lh,r1_xp,r1_x,r1_y,r1_health,r1_max_health,r1_max_mana,r1_level,r1_denies,r1_nearby_creep_death_count,r1_kills,r1_deaths,r1_assists,r1_kda,r1_ability_level,r1_max_hero_hit,r1_gpm_max,r1_gpm_avg,r1_gpm_delta,r1_lhpm_max,r1_lhpm_avg,r1_lhpm_delta,r1_dnpm_max,r1_dnpm_avg,r1_dnpm_delta,r1_xppm_max,r1_xppm_avg,r1_xppm_delta,r1_purchases,r1_buybacks,r1_actions,r1_pings,r1_item_uses,r1_ability_uses,r1_hero_hits,r1_hero_damage,r1_tower_damage,r1_kill_streaks,r1_multi_kills,r1_life_state_0,r1_life_state_1,r1_life_state_2,r1_healing,r1_slots,r1_hero_inventory,r2_hero_id,r2_hero_name,r2_obs_placed,r2_sen_placed,r2_creeps_stacked,r2_camps_stacked,r2_rune_pickups,r2_firstblood_claimed,r2_teamfight_participation,r2_towers_killed,r2_roshans_killed,r2_stuns,r2_randomed,r2_pred_vict,r2_gold,r2_lh,r2_xp,r2_x,r2_y,r2_health,r2_max_health,r2_max_mana,r2_level,r2_denies,r2_nearby_creep_death_count,r2_kills,r2_deaths,r2_assists,r2_kda,r2_ability_level,r2_max_hero_hit,r2_gpm_max,r2_gpm_avg,r2_gpm_delta,r2_lhpm_max,r2_lhpm_avg,r2_lhpm_delta,r2_dnpm_max,r2_dnpm_avg,r2_dnpm_delta,r2_xppm_max,r2_xppm_avg,r2_xppm_delta,r2_purchases,r2_buybacks,r2_actions,r2_pings,r2_item_uses,r2_ability_uses,r2_hero_hits,r2_hero_damage,r2_tower_damage,r2_kill_streaks,r2_multi_kills,r2_life_state_0,r2_life_state_1,r2_life_state_2,r2_healing,r2_slots,r2_hero_inventory,r3_hero_id,r3_hero_name,r3_obs_placed,r3_sen_placed,r3_creeps_stacked,r3_camps_stacked,r3_rune_pickups,r3_firstblood_claimed,r3_teamfight_participation,r3_towers_killed,r3_roshans_killed,r3_stuns,r3_randomed,r3_pred_vict,r3_gold,r3_lh,r3_xp,r3_x,r3_y,r3_health,r3_max_health,r3_max_mana,r3_level,r3_denies,r3_nearby_creep_death_count,r3_kills,r3_deaths,r3_assists,r3_kda,r3_ability_level,r3_max_hero_hit,r3_gpm_max,r3_gpm_avg,r3_gpm_delta,r3_lhpm_max,r3_lhpm_avg,r3_lhpm_delta,r3_dnpm_max,r3_dnpm_avg,r3_dnpm_delta,r3_xppm_max,r3_xppm_avg,r3_xppm_delta,r3_purchases,r3_buybacks,r3_actions,r3_pings,r3_item_uses,r3_ability_uses,r3_hero_hits,r3_hero_damage,r3_tower_damage,r3_kill_streaks,r3_multi_kills,r3_life_state_0,r3_life_state_1,r3_life_state_2,r3_healing,r3_slots,r3_hero_inventory,r4_hero_id,r4_hero_name,r4_obs_placed,r4_sen_placed,r4_creeps_stacked,r4_camps_stacked,r4_rune_pickups,r4_firstblood_claimed,r4_teamfight_participation,r4_towers_killed,r4_roshans_killed,r4_stuns,r4_randomed,r4_pred_vict,r4_gold,r4_lh,r4_xp,r4_x,r4_y,r4_health,r4_max_health,r4_max_mana,r4_level,r4_denies,r4_nearby_creep_death_count,r4_kills,r4_deaths,r4_assists,r4_kda,r4_ability_level,r4_max_hero_hit,r4_gpm_max,r4_gpm_avg,r4_gpm_delta,r4_lhpm_max,r4_lhpm_avg,r4_lhpm_delta,r4_dnpm_max,r4_dnpm_avg,r4_dnpm_delta,r4_xppm_max,r4_xppm_avg,r4_xppm_delta,r4_purchases,r4_buybacks,r4_actions,r4_pings,r4_item_uses,r4_ability_uses,r4_hero_hits,r4_hero_damage,r4_tower_damage,r4_kill_streaks,r4_multi_kills,r4_life_state_0,r4_life_state_1,r4_life_state_2,r4_healing,r4_slots,r4_hero_inventory,r5_hero_id,r5_hero_name,r5_obs_placed,r5_sen_placed,r5_creeps_stacked,r5_camps_stacked,r5_rune_pickups,r5_firstblood_claimed,r5_teamfight_participation,r5_towers_killed,r5_roshans_killed,r5_stuns,r5_randomed,r5_pred_vict,r5_gold,r5_lh,r5_xp,r5_x,r5_y,r5_health,r5_max_health,r5_max_mana,r5_level,r5_denies,r5_nearby_creep_death_count,r5_kills,r5_deaths,r5_assists,r5_kda,r5_ability_level,r5_max_hero_hit,r5_gpm_max,r5_gpm_avg,r5_gpm_delta,r5_lhpm_max,r5_lhpm_avg,r5_lhpm_delta,r5_dnpm_max,r5_dnpm_avg,r5_dnpm_delta,r5_xppm_max,r5_xppm_avg,r5_xppm_delta,r5_purchases,r5_buybacks,r5_actions,r5_pings,r5_item_uses,r5_ability_uses,r5_hero_hits,r5_he

In [19]:
df_test_features.to_csv('test_data.csv')